## How To Perform the IQ Blobs Experiment

### Prerequisites
This guide assumes you have a configured `DeviceSetup` as well as `Qubit` objects with assigned parameters. Please see these guides (add links) if you need to create your setup and qubits for the first time. However, you can also run this notebook "as is" using an emulated session. If you are just getting started with the LabOne Q Applications Library, please don't hesitate to reach out to us at info@zhinst.com.

### Background
In this how-to guide, you'll investigate the complex response of qubit readout for different qubit states and display the results in the IQ-plane. The experiment itself is very simple: we prepare the qubit in the one of its basis states - g, e, or f - and perform single-shot measurements. 

Measurements on superconducting qubits such as Transmons are often based on dispersive readout. Here, the qubit (Qubit) itself is not directly probed, instead a resonator (Res), which is coupled capacitively to the qubit, is probed with a measure pulse close to its resonance frequency (see figure below). The amplitude and phase of the returning measure pulse (here transmitted) contain information about the qubit state.

Here we use the effect that the readout resonator is sensitive to the qubit state as depicted by the transmitted amplitude and phase response of the readout resonator below. 

Note: In the dispersive limit, i.e., when the qubit frequency is far detuned from the resonance frequency of the resonator, there is no direct exchange of energy between the two systems. Hence, we minimize back action on the qubit (quantum non-demolition (QND) measurement).

![Transmitted amplitude and phase response of the readout resonator coupled to a qubit in state g and state e. When the resonator is probed, the transmitted amplitude and phase contain information about the qubit state.](../../../images/dispersive_readout.svg "Transmitted amplitude and phase response of the readout resonator coupled to a qubit in state g and state e. When the resonator is probed, the transmitted amplitude and phase contain information about the qubit state.")

### Measurements in the IQ-plane

In the `iq_blobs` experiment we prepare the qubit in the one of its basis states - e.g. g or e - and perform single-shot measurements. 
For each single-shot measurement we demodulate and integrate the resonator response to obtain the in-phase ( $I$ ) and quadrature ( $Q$ ) component (link to qubit_readout explanation). Amplitude $A$ and phase $\theta$ of a single-shot measurement are directly related to in-phase and quadrature components via 

$$ I = \text{Re}[A\cdot \exp(i\cdot \theta)] $$

$$ Q = \text{Im}[A\cdot \exp(i\cdot \theta)] $$

By repeating the single-shot measurements many times, we obtain the state distribution for each state in the complex plane. Below a schematic complex plane representation of states $g$ and $e$, composed of the in-phase ( $I$ ) and quadrature ( $Q$ ) components. 

![Complex plane representation of states g and e composed of the in-phase (I) and quadrature (Q) components of the transmitted amplitude and phase response of the readout resonator.](../../../images/dispersive_readout_2.svg "Complex plane representation of states g and e composed of the in-phase (I) and quadrature (Q) components of the transmitted amplitude and phase response of the readout resonator.")

A good starting point for the readout frequency is given by the readout resonator frequency with the qubit in the ground state $g$.  






### Import 

You'll start by importing the `iq_blobs` experiment from `laboneq_applications`, as well as `laboneq.simple` and a demo QPU and device setup to run in emulation mode.

In [ ]:
from laboneq.contrib.example_helpers.plotting.plot_helpers import plot_simulation
from laboneq.simple import *

from laboneq_applications.experiments import iq_blobs
from laboneq_applications.qpu_types.tunable_transmon.demo_qpus import demo_platform

### QPU and Device Setup

You'll generate six qubits with pre-defined parameters, as well as a `Device_Setup` consisting of a SHFQC+, HDAWG, and PQSC. If you already have your own `DeviceSetup` and qubits configured, you'll instead initialize the session using your setup.

In [ ]:
my_platform = demo_platform(6)

Then, you'll connect to the `Session`. Here we connect to an emulated one:

In [ ]:
session = Session(my_platform.setup)
session.connect(do_emulation=True)

### Running the IQ blob Workflow

You'll now make the experiment workflow and run:

In [ ]:
# our qubits live here in the demo setup:
qubits = my_platform.qpu.qubits

options = iq_blobs.experiment_workflow.options()
my_workflow = iq_blobs.experiment_workflow(
    session,
    qpu = my_platform.qpu,
    qubits = [qubits[0],qubits[1]],
    states = "gef",
    options=options,
)

result = my_workflow.run()

#### Output Simulation

You can also inspect the compiled experiment and plot the simulated output:

In [ ]:
plot_simulation(
    result.tasks["compile_experiment"].output,
    signal_names_to_show=["drive", "measure"],
    start_time=0,
    length=12e-6,
)

#### Inspecting the Source Code

You can inspect the source code of the `create_experiment` task defined in `iq_blobs` to see how the experiment pulse sequence is created:

In [ ]:
iq_blobs.create_experiment.src

### Changing the Options

We can give our `iq_blobs` experiment options. First, inspect what they currently are:

In [ ]:
my_new_opts = iq_blobs.experiment_workflow.options()
my_new_opts

Then provide new options. This time, you'll change the counts.

In [ ]:
my_new_opts.count(2048)
my_new_opts

#### Run the workflow with updated options

Now, run the workflow with new options and inspect the simulated output.

In [ ]:
my_new_workflow = iq_blobs.experiment_workflow(
    session=session,
    qpu=my_platform.qpu,
    qubits=[qubits[0], qubits[1]],
    states = "gef",
    options=my_new_opts,
)

my_new_results = my_new_workflow.run()
new_compiled_iq_blobs = my_new_results.tasks["compile_experiment"].output
plot_simulation(new_compiled_iq_blobs, length=12e-6)

### Temporary settings

The qubit parameters are used to control the settings of pulses and instruments during the experiment. We can run the IQ blobs experiment with different settings by passing it a copy of the qubits with modified parameters:

In [ ]:
# Make a copy of the qubits
temp_qubits = my_platform.qpu.copy_qubits()
# Change the length of the drive pulses
temp_qubits[0].parameters.ge_drive_length = 1000e-9
temp_qubits[1].parameters.ge_drive_length = 1000e-9
temp_qubits[0].parameters.ef_drive_length = 1000e-9
temp_qubits[1].parameters.ef_drive_length = 1000e-9

my_new_workflow = iq_blobs.experiment_workflow(
    session=session,
    qpu=my_platform.qpu,
    qubits=[temp_qubits[0], temp_qubits[1]],  # pass temporary qubits
    states="gef",
    options=my_new_opts,
)

my_new_results = my_new_workflow.run()
new_compiled_iq_blobs = my_new_results.tasks["compile_experiment"].output
plot_simulation(new_compiled_iq_blobs, length=10e-6)

Great! You've now run your IQ blobs experiment. Check out these other experiments to keep characterizing your qubits:

In [ ]:
# TODO: Add experiment links
# TODO: Add Analysis